In [2]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
data = pd.read_csv('data.csv', index_col=0)

In [4]:
data.drop(['Meal', 'Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'], axis=1, inplace=True)

In [5]:
data

,Date,DaySin,DayCos,MealNum,Item1Num,Item2Num,Item3Num,Item4Num,Item5Num,Item6Num,Item7Num,Item8Num
0,08/08/22,0.000000,1.000000,0,0,0,0,0,0,0,0,0
1,08/08/22,0.000000,1.000000,1,1,1,1,1,1,1,1,-1
2,08/08/22,0.000000,1.000000,2,2,2,-1,-1,-1,-1,-1,-1
3,08/08/22,0.000000,1.000000,3,3,3,2,2,2,2,2,1
4,09/08/22,0.781831,0.623490,0,4,4,3,3,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1339,15/07/23,-0.974928,-0.222521,3,184,245,7,26,16,1,2,-1
1340,16/07/23,-0.781831,0.623490,0,0,53,201,79,0,3,0,0
1341,16/07/23,-0.781831,0.623490,1,15,36,88,27,16,7,1,-1
1342,16/07/23,-0.781831,0.623490,2,234,25,-1,-1,-1,-1,-1,-1


In [6]:
data['Date'] = pd.to_datetime(data['Date'])
data.drop(['Date'], axis=1, inplace=True)
numarr = data.to_numpy()

/var/folders/82/rllbqfts1ngcl7f8ts__f5000000gn/T/ipykernel_12327/275728622.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'])


In [7]:
type(numarr[0][0])

numpy.float64

In [8]:
#for arr in numarr:
#    arr[0] = arr[0].to_numpy()

In [9]:
numarr.shape

(1344, 11)

In [10]:
X_train = numarr[:int(numarr.shape[0]*0.8)]
X_train.shape

(1075, 11)

In [11]:
X_test = numarr[int(numarr.shape[0]*0.8):]
X_test.shape

(269, 11)

In [12]:
X_train_list = []
Y_train_list = []
hops = 28
no_records = X_train.shape[0]

for i in range(hops,no_records):
    X_train_list.append(X_train[i-hops:i])
    Y_train_list.append(X_train[i])

X_train_list, Y_train_list = np.array(X_train_list), np.array(Y_train_list)

In [13]:
X_train_shaped = np.reshape(X_train_list, (X_train_list.shape[0], X_train_list.shape[1], X_train_list.shape[2]))
Y_train_shaped = np.reshape(Y_train_list, (Y_train_list.shape[0], Y_train_list.shape[1]))

In [14]:
X_train_list.shape, Y_train_list.shape

((1047, 28, 11), (1047, 11))

In [15]:
X_train_shaped.shape

(1047, 28, 11)

In [16]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import tensorflow as tf

In [17]:
class MultidimensionalLSTM(tf.keras.Model):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        self.lstm = keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dense = keras.layers.Dense(output_size)
    
    def call(self, inputs):
        x = self.lstm(inputs)
        x = self.dense(x[:, -1, :])
        return x

In [18]:
model = MultidimensionalLSTM(hidden_size=32, output_size=11)
model.compile(optimizer="adam", loss="mse")

In [19]:
model.fit(X_train_shaped, Y_train_shaped, epochs=10000)

Epoch 1/10000
33/33 [==============================] - 2s 7ms/step - loss: 2767.0508
Epoch 2/10000
33/33 [==============================] - 0s 6ms/step - loss: 2734.8857
Epoch 3/10000
33/33 [==============================] - 0s 6ms/step - loss: 2695.7173
Epoch 4/10000
33/33 [==============================] - 0s 6ms/step - loss: 2632.5764
Epoch 5/10000
33/33 [==============================] - 0s 6ms/step - loss: 2554.1047
Epoch 6/10000
33/33 [==============================] - 0s 7ms/step - loss: 2487.3308
Epoch 7/10000
33/33 [==============================] - 0s 7ms/step - loss: 2428.3940
Epoch 8/10000
33/33 [==============================] - 0s 7ms/step - loss: 2370.9307
Epoch 9/10000
33/33 [==============================] - 0s 7ms/step - loss: 2320.9958
Epoch 10/10000
33/33 [==============================] - 0s 7ms/step - loss: 2279.5706
Epoch 11/10000
33/33 [==============================] - 0s 6ms/step - loss: 2241.2029
Epoch 12/10000
33/33 [==============================] - 0s 6ms/

KeyboardInterrupt: 

In [ ]:
X_test.shape

(320, 11)

In [ ]:
hops = 28
no_records = X_test.shape[0]
X_test_list = []
for i in range(hops,no_records):
    X_test_list.append(X_test[i-hops:i])

X_test_list = np.array(X_test_list)
X_test_shaped = np.reshape(X_test_list, (X_test_list.shape[0], X_test_list.shape[1], X_test_list.shape[2]))
X_test_shaped.shape

(292, 28, 11)

In [ ]:
Y_test = model.predict(X_test_shaped)

10/10 [==============================] - 0s 3ms/step


2023-07-13 12:49:31.465424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-13 12:49:31.466495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-13 12:49:31.467710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
Y_test

array([[-0.06761415,  0.01626509,  0.400939  , ..., 10.505636  ,
        -0.2613052 , -0.8870282 ],
       [-0.05110481,  0.5250513 ,  2.3328457 , ..., -0.382154  ,
         6.91089   , 13.112932  ],
       [ 0.06758912, -0.04278363,  1.9145662 , ..., -1.1633902 ,
        -1.2249805 , -1.3215911 ],
       ...,
       [-0.29646865,  0.19351766,  1.7236128 , ...,  8.103509  ,
         2.6470752 ,  2.7854362 ],
       [-0.494512  , -0.1980438 ,  1.5218685 , ...,  0.73517513,
         0.25414658, -4.283777  ],
       [ 0.31667835,  0.69313085,  1.9833537 , ...,  3.4063232 ,
         3.5953965 , 13.450698  ]], dtype=float32)

In [ ]:
pd.DataFrame(Y_test).to_csv('Y_test.csv')